In [1]:
import glob
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array

In [5]:
train_img_names = glob.glob('./data/train/*')
print(type(train_img_names))
print(len(train_img_names))

<class 'list'>
25000


'./data/train\\cat.0.jpg'

In [9]:
demo_img_0 = train_img_names[0]
demo_img_0

'./data/train\\cat.0.jpg'

In [7]:
IMG_DIM = (150, 150)

In [11]:
train_imgs_raw = img_to_array(load_img(demo_img_0, target_size = IMG_DIM))
print(type(train_imgs_raw))
print(train_imgs_raw.shape)

<class 'numpy.ndarray'>
(150, 150, 3)


In [12]:
train_imgs_raw.max()

255.0

In [13]:
train_imgs_raw.min()

0.0

In [15]:
rescale_coef = np.divide(1, train_imgs_raw.max())
rescale_coef

0.00392156862745098

In [16]:
train_data_gen = ImageDataGenerator(
                                    rescale = rescale_coef, shear_range = 0.2,
                                    zoom_range = 0.2, horizontal_flip = True,
                                    vertical_flip = True
                                   )

In [17]:
test_data_gen = ImageDataGenerator(rescale = rescale_coef)

In [18]:
train_generator = train_data_gen.flow_from_directory('./data/train', target_size = IMG_DIM, batch_size = 32, class_mode = 'binary')

Found 0 images belonging to 0 classes.


In [19]:
test_generator = train_data_gen.flow_from_directory('./data/test', target_size = IMG_DIM, batch_size = 32, class_mode = 'binary')

Found 0 images belonging to 0 classes.
